# Train XGBoost Model

In [1]:
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.xgboost
from mlflow.models import infer_signature
import dotenv
import os

In [2]:
import xgboost as xgb
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

print('Python version:', os.sys.version)
print('XGBoost version:', xgb.__version__)
print('Numpy version:', np.__version__)
print('Pandas version:', pd.__version__)

Python version: 3.12.11 (main, Jun 23 2025, 18:10:22) [Clang 17.0.0 (clang-1700.0.13.3)]
XGBoost version: 1.7.6
Numpy version: 2.3.2
Pandas version: 2.3.1


## Configure

In [3]:
dotenv.load_dotenv()

True

In [4]:
MLFLOW_EXPERIMENT = "iris_classification"

## Load Data

In [5]:
iris = datasets.load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train & Log Model

In [6]:
mlflow.set_experiment(MLFLOW_EXPERIMENT)

with mlflow.start_run():
    print("Starting MLflow run...")
    params = {
        "n_estimators": 100,
        "learning_rate": 0.1,
        "max_depth": 3,
        "random_state": 42
    }
    mlflow.log_params(params)

    print("Training model...")
    model = XGBClassifier(**params, verbosity=0) 
    model.fit(X_train, y_train)

    print("Evaluating model...")
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)
    print(f"Model accuracy: {accuracy:.4f}")

    print("Logging model to MLflow...")
    signature = infer_signature(X, y_pred)
    model_info = mlflow.xgboost.log_model(model, "model", signature=signature)
    

Starting MLflow run...
Training model...
Evaluating model...


2025/07/30 14:16:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Model accuracy: 1.0000
Logging model to MLflow...
🏃 View run incongruous-dove-800 at: http://localhost:8080/#/experiments/845806305527103591/runs/9b87915973ab4535a01a483c80bde036
🧪 View experiment at: http://localhost:8080/#/experiments/845806305527103591
🏃 View run incongruous-dove-800 at: http://localhost:8080/#/experiments/845806305527103591/runs/9b87915973ab4535a01a483c80bde036
🧪 View experiment at: http://localhost:8080/#/experiments/845806305527103591


In [7]:
# Test predictions
X_test_sample = X_test.sample(5, random_state=42)
print('Test sample:')
display(X_test_sample)
print('Predictions:')
model.predict(X_test_sample)  

Test sample:


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
143,6.8,3.2,5.9,2.3
56,6.3,3.3,4.7,1.6
128,6.4,2.8,5.6,2.1
69,5.6,2.5,3.9,1.1
68,6.2,2.2,4.5,1.5


Predictions:


array([2, 1, 2, 1, 1])